In [63]:
# Load the packages
import pandas as pd
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt # visualizing data
import seaborn as sns 

In [64]:
df = pd.read_csv('C:/Users/DELL/Desktop/[RBAC 2023] Round 2 Dataset.csv')

In [65]:
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])

last_date = df['TransactionDate'].max()

last_month = last_date.to_period('M')

df_filtered = df[df['TransactionDate'].dt.to_period('M') != last_month]

df = df_filtered

C:\Users\DELL\AppData\Local\Temp\ipykernel_14712\2015237899.py:5: UserWarning: Converting to Period representation will drop timezone information.
  last_month = last_date.to_period('M')
C:\Users\DELL\anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:1143: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [66]:
df

,Unnamed: 0,BillID,Channel,OrderFrom,TransactionDate,SalesAmount,CustomerID,CustomerGender,VoucherStatus,Province
0,0,0,Take Away,CALL CENTER,2021-10-01 00:00:00+00:00,296891.0,1753863,Unknown,No,Ho Chi Minh City
1,1,1,Take Away,STORE,2021-10-01 00:00:00+00:00,301782.0,1124050,Unknown,No,Hanoi
2,2,2,Take Away,WEBSITE,2021-10-01 00:00:00+00:00,319792.0,1626827,Male,No,Hanoi
3,3,3,Take Away,STORE,2021-10-01 00:00:00+00:00,424762.0,125643,Male,No,Hanoi
4,4,4,Delivery,STORE,2021-10-01 00:00:00+00:00,280031.0,2117237,Unknown,No,Hanoi
...,...,...,...,...,...,...,...,...,...,...
1397197,1397197,1397197,Delivery,WEBSITE,2023-06-30 00:00:00+00:00,307336.0,1526475,Female,Yes,Hanoi
1397198,1397198,1397198,Take Away,STORE,2023-06-30 00:00:00+00:00,210907.0,48613,Female,No,Hanoi
1397199,1397199,1397199,Take Away,STORE,2023-06-30 00:00:00+00:00,237248.0,711050,Unknown,No,Hanoi
1397200,1397200,1397200,Delivery,CALL CENTER,2023-06-30 00:00:00+00:00,119985.0,2046935,Female,No,Ho Chi Minh City


In [67]:
# Calculate the Recency of the Customer
from datetime import datetime

df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])

df['TransactionDate'] = df['TransactionDate'].dt.tz_localize(None)

# print(df['TransactionDate'])

recent_purchase = df.groupby('CustomerID')['TransactionDate'].max()

now = datetime.now().replace(tzinfo=None)
recency = now - recent_purchase
recency = recency.dt.days

recency_df = pd.DataFrame({'CustomerID': recent_purchase.index, 'Recency': recency})

print(recency_df)

            CustomerID  Recency
CustomerID                     
0                    0      687
1                    1      614
8                    8      248
14                  14      168
16                  16      421
...                ...      ...
2174066        2174066      312
2174072        2174072      342
2174073        2174073      272
2174075        2174075      336
2174083        2174083      756

[718050 rows x 2 columns]


In [68]:
# Calculate the Frequency of the customers
frequency = df.groupby('CustomerID')['TransactionDate'].count()

frequency_df = pd.DataFrame({'CustomerID': frequency.index, 'Frequency': frequency})

print(frequency_df)

            CustomerID  Frequency
CustomerID                       
0                    0          1
1                    1          1
8                    8          2
14                  14          1
16                  16          1
...                ...        ...
2174066        2174066          1
2174072        2174072          3
2174073        2174073          1
2174075        2174075          1
2174083        2174083          1

[718050 rows x 2 columns]


In [69]:
# Monetary of the customer
total_spent_per_customer = df.groupby('CustomerID')['SalesAmount'].sum()

print(total_spent_per_customer)

CustomerID
0           411663.0
1           105503.0
8          1941696.0
14          267607.0
16          286558.0
             ...    
2174066     143685.0
2174072     953618.0
2174073     514503.0
2174075     262489.0
2174083     185271.0
Name: SalesAmount, Length: 718050, dtype: float64


In [70]:
# RFM Table"
RMF_Table = pd.DataFrame({'Recency': recency, 'Frequency': frequency, 'Monetary': total_spent_per_customer})

print(RMF_Table)

            Recency  Frequency   Monetary
CustomerID                               
0               687          1   411663.0
1               614          1   105503.0
8               248          2  1941696.0
14              168          1   267607.0
16              421          1   286558.0
...             ...        ...        ...
2174066         312          1   143685.0
2174072         342          3   953618.0
2174073         272          1   514503.0
2174075         336          1   262489.0
2174083         756          1   185271.0

[718050 rows x 3 columns]


In [71]:
quantiles = RMF_Table.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()
quantiles

{'Recency': {0.25: 265.0, 0.5: 440.0, 0.75: 607.0},
 'Frequency': {0.25: 1.0, 0.5: 1.0, 0.75: 2.0},
 'Monetary': {0.25: 200367.0, 0.5: 318378.0, 0.75: 605750.75}}

In [72]:
# RFM Segmentation
rfmSegmentation = RMF_Table

# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1
    
rfmSegmentation['R_Quartile'] = rfmSegmentation['Recency'].apply(RClass, args=('Recency',quantiles,))
rfmSegmentation['F_Quartile'] = rfmSegmentation['Frequency'].apply(FMClass, args=('Frequency',quantiles,))
rfmSegmentation['M_Quartile'] = rfmSegmentation['Monetary'].apply(FMClass, args=('Monetary',quantiles,))

rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

rfmSegmentation.head()
                                                                                       

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMClass
CustomerID,,,,,,,
0,687,1,411663.0,4,4,2,442
1,614,1,105503.0,4,4,4,444
8,248,2,1941696.0,1,2,1,121
14,168,1,267607.0,1,4,3,143
16,421,1,286558.0,2,4,3,243


In [73]:
rfmSegmentation[rfmSegmentation['RFMClass'] == '111'].sort_values('Monetary', ascending=False)


,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMClass
CustomerID,,,,,,,
1746973,173,13,124945291.0,1,1,1,111
175175,147,19,115319448.0,1,1,1,111
488818,179,17,102975171.0,1,1,1,111
1836825,158,89,77005096.0,1,1,1,111
1868836,200,83,74377341.0,1,1,1,111
...,...,...,...,...,...,...,...
111482,163,3,605925.0,1,1,1,111
2107370,161,3,605915.0,1,1,1,111
544007,215,3,605911.0,1,1,1,111


In [74]:
rfmSegmentation[rfmSegmentation['RFMClass'] == '444'].sort_values('Monetary', ascending=False)


,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMClass
CustomerID,,,,,,,
1719728,762,1,200367.0,4,4,4,444
1792117,651,1,200366.0,4,4,4,444
662978,727,1,200366.0,4,4,4,444
728996,632,1,200365.0,4,4,4,444
784694,615,1,200363.0,4,4,4,444
...,...,...,...,...,...,...,...
930531,620,1,12481.0,4,4,4,444
1487864,712,1,11711.0,4,4,4,444
1806292,687,1,11637.0,4,4,4,444


In [76]:
def segment(value):
	if value =='111':
    	return 'High'
    elif value == '444':
    	return 'Low'
    else:
    	return 'Mid'

rfmSegmentation['Class']=rfmSegmentation['RFMClass'].apply(segment)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 4)

In [ ]:
# Calculate the monthly/quarterly churn rate
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])

df['Period'] = df['TransactionDate'].dt.to_period('M')

first_transaction = df.groupby('CustomerID')['TransactionDate'].min().dt.to_period('M')

unique_customers = df.groupby('Period')['CustomerID'].nunique()

new_customers = df.groupby(['CustomerID', 'Period']).size().groupby(level=1).count()

churn_rate = pd.Series(dtype=float)

for period in df['Period'].unique():
    begin_customers = unique_customers[unique_customers.index < period].sum()
    end_customers = unique_customers[unique_customers.index <= period].sum()
    new_customers_count = new_customers[new_customers.index == period].sum()

    if begin_customers > 0:
        churn = (end_customers - begin_customers - new_customers_count) / begin_customers
        churn_rate.at[period] = churn
    else:
        churn_rate.at[period] = 0

print(churn_rate)

In [ ]:
# 4 chart riêng tính tổng sales amount breakdown của 4 mục gender/region/channel/platform dao động theo tháng
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
df['Month'] = df['TransactionDate'].dt.strftime('%Y-%m')

categories = ['CustomerGender', 'Province', 'OrderFrom', 'Channel']

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Monthly Sales Amount by Categories')

for ax, category in zip(axes.flatten(), categories):
    monthly_sales = df.groupby(['Month', category])['SalesAmount'].sum().reset_index()

    sns.lineplot(data=monthly_sales, x='Month', y='SalesAmount', hue=category, ax=ax)
    ax.set_title(f'Monthly Sales Amount by {category}')
    ax.set_ylabel('Sales Amount')
    ax.set_xlabel('Month')
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.subplots_adjust(top=0.9)

plt.show()


In [ ]:
# Monthly/Quarterly total number customer
import pandas as pd

df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])

df['Month'] = df['TransactionDate'].dt.to_period('M')
customers_by_month = df.groupby('Month')['CustomerID'].nunique()

df['Quarter'] = df['TransactionDate'].dt.to_period('Q')
customers_by_quarter = df.groupby('Quarter')['CustomerID'].nunique()

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
customers_by_month.plot(kind='bar')
plt.title('Customers by Month')
plt.xlabel('Month')
plt.ylabel('Number of Customers')

plt.subplot(1, 2, 2)
customers_by_quarter.plot(kind='bar')
plt.title('Customers by Quarter')
plt.xlabel('Quarter')
plt.ylabel('Number of Customers')

plt.tight_layout()
plt.show()

In [ ]:
categories = ['CustomerGender', 'OrderFrom', 'Channel', 'Province']

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Unique Customers by Category')

for ax, category in zip(axes.flatten(), categories):
    customer_count = df.groupby(category)['CustomerID'].nunique()

    ax.pie(customer_count, labels=customer_count.index, autopct='%1.1f%%', startangle=140)
    ax.set_title(f'Unique Customers by {category}')

plt.tight_layout()
plt.subplots_adjust(top=0.9)

plt.show()

In [ ]:
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
df['Month'] = df['TransactionDate'].dt.to_period('M')
df['Quarter'] = df['TransactionDate'].dt.to_period('Q')

categories = ['CustomerGender', 'Province', 'Channel', 'OrderFrom']

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Number of Bills by Category - Monthly Breakdown')

for ax, category in zip(axes.flatten(), categories):
    monthly_data = df.groupby(['Month', category]).size().unstack().fillna(0)
    
    monthly_data.plot(kind='bar', stacked=True, ax=ax)
    ax.set_title(f'Number of Bills by {category} - Monthly')
    ax.set_ylabel('Number of Bills')
    ax.set_xlabel('Month')
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Number of Bills by Category - Quarterly Breakdown')

for ax, category in zip(axes.flatten(), categories):
    quarterly_data = df.groupby(['Quarter', category]).size().unstack().fillna(0)
    
    quarterly_data.plot(kind='bar', stacked=True, ax=ax)
    ax.set_title(f'Number of Bills by {category} - Quarterly')
    ax.set_ylabel('Number of Bills')
    ax.set_xlabel('Quarter')
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()


In [ ]:
categories = ['CustomerGender', 'Province', 'Channel', 'OrderFrom']

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Number of Bills by Category')

for ax, category in zip(axes.flatten(), categories):
    bill_count = df.groupby(category).size()

    ax.pie(bill_count, labels=bill_count.index, autopct='%1.1f%%', startangle=140)
    ax.set_title(f'Number of Bills by {category}')

plt.tight_layout()
plt.subplots_adjust(top=0.9)

plt.show()

In [ ]:
bills_per_customer = df.groupby('CustomerID').size()

percentiles = [0.25, 0.5, 0.75, 0.95, 0.99] 
percentile_values = bills_per_customer.quantile(percentiles)

percentile_data = pd.DataFrame({'Percentile': percentiles, 'Number of Bills': percentile_values.values})

plt.figure(figsize=(10, 6))
plt.bar(x=percentile_data['Percentile'].astype(str), height=percentile_data['Number of Bills'])
plt.title('Percentiles for Number of Bills per Customer')
plt.xlabel('Percentile')
plt.ylabel('Number of Bills')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Average Frequency of Customers
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
df['Month'] = df['TransactionDate'].dt.to_period('M')
df['Quarter'] = df['TransactionDate'].dt.to_period('Q')

monthly_frequency = df.groupby(['CustomerID', 'Month']).size().groupby('Month').mean()

quarterly_frequency = df.groupby(['CustomerID', 'Quarter']).size().groupby('Quarter').mean()

plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
monthly_frequency.plot(kind='line')
plt.title('Average Monthly Frequency of Customers')
plt.xlabel('Month')
plt.ylabel('Average Frequency')

plt.subplot(1, 2, 2)
quarterly_frequency.plot(kind='line')
plt.title('Average Quarterly Frequency of Customers')
plt.xlabel('Quarter')
plt.ylabel('Average Frequency')

plt.tight_layout()
plt.show()


In [ ]:
# %Voucher use in month, quarter.
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
df['Month'] = df['TransactionDate'].dt.to_period('M')
df['Year'] = df['TransactionDate'].dt.year

df['VoucherUsed'] = df['VoucherStatus'].apply(lambda x: 1 if x == 'Yes' else 0)

monthly_voucher_use = df.groupby('Month')['VoucherUsed'].mean() * 100

yearly_voucher_use = df.groupby('Year')['VoucherUsed'].mean() * 100

# Plot
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
monthly_voucher_use.plot(kind='line')
plt.title('Percentage of Voucher Usage per Month')
plt.xlabel('Month')
plt.ylabel('Percentage of Voucher Usage (%)')

plt.subplot(1, 2, 2)
yearly_voucher_use.plot(kind='line')
plt.title('Percentage of Voucher Usage per Year')
plt.xlabel('Year')
plt.ylabel('Percentage of Voucher Usage (%)')

plt.tight_layout()
plt.show()
